In [1]:
# get our db url
# create db engine via sqlalchemy

In [2]:
from env import config, BASE_DIR
from sqlalchemy import create_engine, text

In [3]:
import gzip
import pandas as pd # dask

In [4]:
DATABASE_URL = config("FLIGHT_PRICES_DATABASE_URL", cast=str, default=None)
DATABASE_URL is not None

True

In [5]:
engine = create_engine(str(DATABASE_URL))

In [6]:
DATASET_DIR = BASE_DIR / "datasets"
DATASET_DIR.exists()

True

In [7]:
GRETEL_DIR = DATASET_DIR / "gretel"
GRETEL_DIR.exists()

True

In [8]:
table_name = "flight_prices"
df = pd.read_sql(table_name, engine, index_col="index")
df.tail(n=1)
# out_df.to_sql(table_name, engine, if_exists='replace', index=True)

,flightDate,startingAirport,destinationAirport,isBasicEconomy,isRefundable,isNonStop,segmentsAirlineName,totalFare
index,,,,,,,,
9591,2022-04-22,IAD,LAX,1,0,1,United,221


In [9]:
last_row = df.tail(n=1)
last_idx = last_row.index[0]
last_idx

9591

In [10]:
max_flight_date = df.flightDate.max() # .date()
max_flight_date

Timestamp('2022-04-22 00:00:00')

In [11]:
# list(GRETEL_DIR.glob("*.gz"))

In [36]:
import datetime

In [41]:
gretel_dfs = []
for path in GRETEL_DIR.glob("*.gz"):
    with gzip.open(path) as f:
        gretel_dfs.append(pd.read_csv(f))
new_df = pd.concat(gretel_dfs)
new_df.index = new_df.index + last_idx + 1 
new_df['flightDate'] = pd.to_datetime(new_df['flightDate'])
new_df['flightDate'] = new_df['flightDate'].apply(lambda x: max_flight_date + (max_flight_date - x) + datetime.timedelta(days=1))
# del new_df['totalFare']
new_df.head()

,flightDate,startingAirport,destinationAirport,isBasicEconomy,isRefundable,isNonStop,segmentsAirlineName,totalFare
9592,2022-04-27,SFO,JFK,0,0,1,United,362
9593,2022-04-24,LAX,ORD,0,0,0,Delta||Delta,181
9594,2022-04-28,ORD,OAK,0,0,0,Spirit Airlines||Spirit Airlines,421
9595,2022-04-25,SFO,IAD,0,0,1,American Airlines,241
9596,2022-04-27,SFO,EWR,0,0,1,JetBlue Airways,261


In [42]:
new_df.flightDate.max().date()

datetime.date(2022, 4, 28)

In [46]:
new_df.to_sql(table_name, engine, if_exists='append', index=True)

10000

In [49]:
df = pd.read_sql(table_name, engine, index_col="index")
df.head()

,flightDate,startingAirport,destinationAirport,isBasicEconomy,isRefundable,isNonStop,segmentsAirlineName,totalFare
index,,,,,,,,
0,2022-04-17,ATL,BOS,0,0,0,American Airlines||American Airlines,273
1,2022-04-17,ATL,BOS,0,0,0,Spirit Airlines||Spirit Airlines,301
2,2022-04-17,ATL,BOS,0,0,0,United||United,352
3,2022-04-17,ATL,BOS,0,0,1,American Airlines,355
4,2022-04-17,ATL,BOS,0,0,1,Delta,307
